In [1]:
import dash
import dash_core_components as dcc 
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px
import statsmodels.api as sm
import pandas as pd 

In [2]:
import os
os.getcwd()

'/Users/siqisun/Documents/GitHub/CMM'

In [30]:
df = pd.read_csv('summary.csv')
df_filt = df.dropna(subset=["pa_approved"])

provider = df['bin'].unique()
drug = df['drug'].unique()
year = df['calendar_year'].unique()

tdf = df_filt[["calendar_year", "calendar_month"]].drop_duplicates()
tdf['year-month'] = tdf["calendar_year"].map(str)+'-'+tdf['calendar_month'].map(str)
tdf = tdf.set_index(["calendar_year", "calendar_month"])
tdf['rate'] = 0

In [36]:
app = dash.Dash()
app.layout = html.Div([
    html.Div([
        html.Label('Drug_type'),
        html.Div([
            dcc.RadioItems(
                id='drug_type',
                options=[{'label': i, 'value': i} for i in drug],
                value=drug[0],
                labelStyle={'display': 'inline-block'})
        ]),
        html.Label('Provider_type'),
        html.Div([
            dcc.RadioItems(
                id='provider_type',
                options=[{'label': i, 'value': i} for i in provider],
                value=provider[0],
                labelStyle={'display': 'inline-block'})
        ]),
        html.Label('Approval Rate across time'),
    ]),
    html.Div([
        dcc.Graph(id='app_rate')
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}), 
])

@app.callback(
    dash.dependencies.Output('app_rate', 'figure'),
    [dash.dependencies.Input('drug_type','value'),
     dash.dependencies.Input('provider_type','value')]
)

def update_app_rate(value_drug, value_provider):    
    for j in year:
        df_cal = df_filt[(df_filt['drug']==value_drug)&(df_filt['bin']==value_provider)&(df_filt['calendar_year']==j)]
        for i in range(1, 13):
            PA_requests = len(df_cal[df_cal['calendar_month']==i])
            PA_approved = len(df_cal[(df_cal['calendar_month']==i)&(df_cal['pa_approved']==1)])        
            PA_approval_rate = (PA_approved/PA_requests)*100
            tdf.loc[(j, i), "rate"] = PA_approval_rate

    fig = px.line(tdf, x=tdf["year-month"], y=tdf["rate"])
    return fig

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/May/2021 15:02:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:02:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:02:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:02:05] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:02:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:02:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:02:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:04:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:04:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:21:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 15:41:01] "POST /_dash-update-component HTTP/1.1" 200 -
